# 고장진단 머신러닝 모델
데이터 파일 가공/머신러닝 모델 생성/성능평가 함수 포함

In [ ]:
from google.colab import drive
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import math
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor


In [ ]:
drive.mount('/content/drive')

In [ ]:
#체계단위 
def Process_File(TestFile):
    dataRaw = pd.read_csv(TestFile)

    dataCol = dataRaw.columns
    dataRaw = dataRaw.drop(['IMU_Bit'],axis = 1 )
    dataRaw = dataRaw.reset_index()
    dataRaw.columns = dataCol
    dataRaw = dataRaw[dataRaw['IMU_MODE'] == 8]
    dataRaw = dataRaw.reset_index(drop=True)

    dataSet = pd.DataFrame()
    dataSet['M'] = dataRaw['IMU_MODE']

    # XW YW ZW => [rad/s] XYZ_DEL_ANG => [mili deg/s]
    dataSet[['XW','YW','ZW']] = dataRaw[['X_DEL_ANG','Y_DEL_ANG','Z_DEL_ANG']] * 18000 / math.pi

    dataSet[['R','P','Y']] = dataRaw[['ROLL','PITCH','HEADING']]

    #p q r => [deg/s]
    dataSet['p'] = np.rad2deg(-dataSet['XW'])
    dataSet['q'] = np.rad2deg(-dataSet['ZW'])
    dataSet['r'] = np.rad2deg(dataSet['YW'])

    #100초 동안의 p q r AVG
    p = dataSet['p'].mean()
    q = dataSet['q'].mean()
    r = dataSet['r'].mean()

    R = dataSet['R'].values
    P = dataSet['P'].values
    Y = dataSet['Y'].values

    #오일러 각속도
    dataSet['Rdot'] =  p + ( q*np.sin(np.deg2rad(R)) + r*np.cos(np.deg2rad(R)) ) * np.tan(np.deg2rad(P))
    dataSet['Pdot'] = q*np.cos(np.deg2rad(R)) - r*np.sin(np.deg2rad(R))
    dataSet['Ydot'] = (q*np.sin(np.deg2rad(R)) + r*np.cos(np.deg2rad(R))) / np.cos(np.deg2rad(P))
    dataSet[['Rdot','Pdot','Ydot']] = np.rad2deg(dataSet[['Rdot','Pdot','Ydot']])

    return dataSet

In [ ]:
FullData = []

FullData += glob.glob(r'/content/drive/MyDrive/DataSet/8,9LOT/8LOT/*.csv')
FullData += glob.glob(r'/content/drive/MyDrive/DataSet/8,9LOT/9LOT/*.csv')
FullData += glob.glob(r'/content/drive/MyDrive/DataSet/10,11LOT/10LOT/*.csv')
FullData += glob.glob(r'/content/drive/MyDrive/DataSet/10,11LOT/11LOT/*.csv')

In [ ]:
resultALL = []

for TestFile in FullData :
    try :
        SN = re.search(r"(\d{9})", TestFile).group(1)

        dataSet = Process_File(TestFile)

        dataSetT = pd.DataFrame()
        dataSetT[['R','P','Y']] = dataSet[['R','P','Y']].copy()

        #100초 평균 p q r
        p = dataSet['p'].mean()
        q = dataSet['q'].mean()
        r = dataSet['r'].mean()

        #N초 평균 p q r
        p5 = dataSet[0:200]['p'].mean()
        q5 = dataSet[0:200]['q'].mean()
        r5 = dataSet[0:200]['r'].mean()
        p20 = dataSet[0:2000]['p'].mean()
        q20 = dataSet[0:2000]['q'].mean()
        r20 = dataSet[0:2000]['r'].mean()

        #100초 평균 오일러 각속도
        Rdot, Pdot, Ydot = dataSet[0:10000][['Rdot','Pdot','Ydot']].mean().values

        #N초 평균 오일러 각속도
        Rdot5, Pdot5, Ydot5 = dataSet[0:200][['Rdot','Pdot','Ydot']].mean().values
        Rdot20, Pdot20, Ydot20 = dataSet[0:2000][['Rdot','Pdot','Ydot']].mean().values

        #slope => 각도 기울기 정보
        sl = pd.DataFrame()
        sl[['Rdel5','Pdel5','Ydel5']] = (dataSetT[100:10000].values - dataSetT[0:9900])/1
        sl[['Rdel','Pdel','Ydel']]  = ((dataSetT.iloc[10000] - dataSetT.iloc[0])/100).values

        sl[['Rd5','Pd5','Yd5']]  = sl[0:100][['Rdel5','Pdel5','Ydel5']].mean().values #500번째 index까지만 사용 => 5초 정보
        sl[['Rd20','Pd20','Yd20']]  = sl[0:1900][['Rdel5','Pdel5','Ydel5']].mean().values

        result = [SN] + [p,p5, p20, q,q5,q20, r,r5, r20, Rdot, Pdot, Ydot, Rdot5, Pdot5, Ydot5, Rdot20, Pdot20, Ydot20] + list(sl[['Rd5','Rd20','Rdel','Pd5','Pd20','Pdel','Yd5','Yd20','Ydel']].mean())
        resultALL.append(result)
    except :
        print(SN)
        pass

In [ ]:
col = ['SN','p','p5','p20', 'q','q5','q20', 'r','r5', 'r20', 'Rdot', 'Pdot', 'Ydot','Rdot5', 'Pdot5', 'Ydot5','Rdot20', 'Pdot20', 'Ydot20', 'Rd5','Rd20', 'Rdel','Pd5','Pd20', 'Pdel','Yd5', 'Yd20', 'Ydel']
aly = pd.DataFrame(resultALL, columns = col)
#aly = pd.read_csv(r'/content/drive/MyDrive/DataSet/MyData.csv')

In [ ]:
aly.to_csv(r'/content/drive/MyDrive/DataSet/MyData.csv')

In [ ]:
test = aly[['Rd5','Rdel','Pd5','Pdel','Yd5','Ydel']].round(5)*100
test['SN'] = aly['SN']

threshold = 3.3

test['ClassR'] = 0
test['ClassR'].iloc[abs(test['Rdel']) >= threshold] = 1
test['ClassP'] = 0
test['ClassP'].iloc[abs(test['Pdel']) >= threshold] = 1
test['ClassY'] = 0
test['ClassY'].iloc[abs(test['Ydel']) >= threshold] = 1

aly['ClassY'] = 0
aly['ClassY'].iloc[abs(aly['Ydel']*100) >= threshold] = 1
aly['ClassR'] = 0
aly['ClassR'].iloc[abs(aly['Rdel']*100) >= threshold] = 1
aly['ClassP'] = 0
aly['ClassP'].iloc[abs(aly['Pdel']*100) >= threshold] = 1

In [ ]:
print(test['ClassR'].value_counts())
print(test['ClassP'].value_counts())
print(test['ClassY'].value_counts())

# 머신러닝 모델 생성

In [ ]:
from sklearn.preprocessing import StandardScaler

X_train = np.array(aly[['p','q','r','Rdel','Pdel','Ydel', 'Rdot', 'Pdot', 'Ydot']])
X_test = np.array(aly[['p5','q5','r5','Rd5','Pd5','Yd5','Rdot5','Pdot5','Ydot5']])

ss= StandardScaler()
ss.fit(X_train)
X_train_scaled = ss.transform(X_train)
X_test_scaled = ss.transform(X_test)

y_train = np.array(test['Ydel'])
y_test = np.array(test['Ydel'])

In [ ]:
#선형회귀 모델
modelLRR = LinearRegression()
modelLRR.fit(X_train_scaled, y_train)

test['RhatLR'] = modelLRR.predict(X_test)
test['RerrLR'] = y_test - test['RhatLR']

print("훈련 세트 예측 정확도: {:.3f}".format(modelLRR.score(X_train_scaled, y_train)))
print("테스트 세트 예측 정확도: {:.3f}".format(modelLRR.score(X_test_scaled, y_test)))

In [ ]:
#결정트리 분류 모델
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

treeR = DecisionTreeClassifier(max_depth=3, random_state=0)
treeR.fit(X_train_scaled, y_trainCL)

print("훈련 세트 정확도: {:.3f}".format(treeR.score(X_train_scaled, y_trainCL)))
print("테스트 세트 정확도: {:.3f}".format(treeR.score(X_test_scaled, y_testCL)))

In [ ]:
#랜덤포레스트 모델
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import seaborn as sns
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : range(1, 3, 1),
          'min_samples_split' : range(1,3,1)}

gs = GridSearchCV(RandomForestRegressor(random_state = 42), params, n_jobs = -1)

gs.fit(X_train_scaled, y_train)

rf = gs.best_estimator_

print("훈련 세트 정확도: {:.3f}".format(rf.score(X_train_scaled, y_train)))
print("테스트 세트 정확도: {:.3f}".format(rf.score(X_test_scaled, y_test)))

In [ ]:
#다중회귀 모델

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

poly = PolynomialFeatures(degree=2, include_bias=False)

X_train = np.array(aly[['p','q','r','Rdel','Pdel','Ydel', 'Rdot', 'Pdot', 'Ydot']])
X_test = np.array(aly[['p5','q5','r5','Rd5','Pd5','Yd5','Rdot5','Pdot5','Ydot5']])

X_train = poly.fit_transform(X_train)
X_test = poly.fit_transform(X_test)

ss= StandardScaler()
ss.fit(X_train)
X_train_scaled = ss.transform(X_train)
X_test_scaled = ss.transform(X_test)

y_train = np.array(test['Ydel'])
y_test = np.array(test['Ydel'])

lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

In [ ]:
#성능평가 함수
index = []
R = 3.3
test_size = len(y_test)

def R_ModelScore(Model):
    TP, TN, FP, FN, RMSE = 0, 0, 0, 0, 0
    for i in range(test_size):
        RMSE += np.sqrt((y_test[i] - Model.predict(X_test_scaled)[i])**2)
        if (abs(y_test[i]) >=  R) & (abs(Model.predict(X_test_scaled)[i]) >= R):
            TP += 1
        elif(abs(y_test[i]) < R) & (abs(Model.predict(X_test_scaled)[i]) < R):
            TN += 1
        elif(abs(y_test[i]) < R) & (abs(Model.predict(X_test_scaled)[i]) >= R):
            FP += 1
        elif(abs(y_test[i]) >= R) & (abs(Model.predict(X_test_scaled)[i]) < R):
            FN += 1


    Precision = TP/(TP+FP) if (TP+FP) > 0 else 0
    Recall = TP/(TP+FN) if (TP +FN) > 0 else 0
    Accuracy = (TP+TN)/(TP+TN+FP+FN) if (TP + TN + FP + FN) > 0 else 0
    F1 = 2*Precision*Recall/(Precision+Recall) if (Precision + Recall) > 0 else 0
    RMSE = RMSE/test_size

    print(f"TP: {TP}\nTN: {TN}\nFP: {FP}\nFN: {FN}")
    print(f"Precision: {Precision}, Recall: {Recall}")
    print(f"Accuracy: {Accuracy}, F1: {F1}")
    print(f"RMSE: {RMSE}")

def C_ModelScore(Model):
    TP, TN, FP, FN, RMSE = 0, 0, 0, 0, 0
    for i in range(test_size):
        RMSE += np.sqrt((y_test[i] - Model.predict(X_test_scaled)[i])**2)
        if(y_testCL[i] == 1) & (Model.predict(X_test_scaled)[i] == 1):
            TP += 1
        elif(y_testCL[i] == 0) & (Model.predict(X_test_scaled)[i] == 0):
            TN += 1
        elif(y_testCL[i] == 0) & (Model.predict(X_test_scaled)[i] == 1):
            FP += 1
        elif(y_testCL[i] == 1) & (Model.predict(X_test_scaled)[i] == 0):
            FN += 1

    Precision = TP/(TP+FP) if (TP+FP) > 0 else 0
    Recall = TP/(TP+FN) if (TP +FN) > 0 else 0
    Accuracy = (TP+TN)/(TP+TN+FP+FN) if (TP + TN + FP + FN) > 0 else 0
    F1 = 2*Precision*Recall/(Precision+Recall) if (Precision + Recall) > 0 else 0
    RMSE = RMSE/test_size

    print(f"TP: {TP}\nTN: {TN}\nFP: {FP}\nFN: {FN}")
    print(f"Precision: {Precision}, Recall: {Recall}")
    print(f"Accuracy: {Accuracy}, F1: {F1}")
    print(f"RMSE: {RMSE}")